<a href="https://colab.research.google.com/github/worldbank/dec-python-course/blob/session3/1-foundations/3-numpy-and-pandas/foundations_s3_bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Libraries (Bonus)
If you want to lock/control the versions of all packages used by your script (aka. dependencies), you can put them in a `requirements.txt` file in the format of `package_name==package_version` one package per line, then instruct pip to install the dependences in the `requirements.txt` file with bash command `pip install -r requirements.txt`. Using `requirements.txt` for dependency management is a very common practice in python projects.

## Exercises
The following exercises are meant to be completed step by step. Upon successful completion of all steps, the last cell should execute without any `AssertionError`.
1. Git clone this course repository from github: https://github.com/worldbank/dec-python-course

In [ ]:
# hint: use ! and bash command `git clone`

2. Change your working directory to the folder where this file is in `
dec-python-course/1-foundations/3-numpy-and-pandas/`

In [ ]:
# hint: use % and bash command `cd`

3. Install all python libraries as specified in `requirements.txt`, without copying the content of `requirements.txt`. Upon successful installation, click on "RESTART RUNTIME" in the code cell output.

In [ ]:
# hint: use ! and bash command `pip install` with flag `-r`

4. Import `matplotlib`'s `pyplot` module and alias it as `plt`

In [ ]:
# hint: use `import ... as ...`

# === Do not modify code below ===
fig = plt.figure(figsize=(10, 80))
assert hasattr(fig, "subfigures"),\
 "If correct version of matplotlib were installed you should not see this message"

print("Well done!")